In [1]:
import nibabel as nib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
import math

In [2]:
project_dir = '/home/ROBARTS/myousif/projects_inprogress/hippocharts/'
project_data_dir = os.path.join(project_dir,'data')

In [3]:
all_datasets_path = os.path.abspath('/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/')

datasets_path_names = ['AOMIC/ds002785','AOMIC/ds002790','AOMIC/ds003097','cam-can','hcp1200','hcp_aging','mpi_leipzig','neurocog_aging','narratives','QTIM']
datasets_path_list = [os.path.join(all_datasets_path,dset) for dset in datasets_path_names]

datasets_names = ['aomic_piop1','aomic_piop2','aomic_id1000','cam-can','hcp1200','hcp_aging','mpi_leipzig','neurocog_aging','narratives','qtim']
datasets_paths = dict(zip(datasets_names,datasets_path_list))

In [4]:
hippunfold_path = lambda direc: os.path.join(os.path.abspath(direc),'derivatives/hippunfold/hippunfold')
diff_path = lambda direc, metric='FA': os.path.join(os.path.abspath(direc),f'derivatives/hipposample')

In [5]:
master_participant_df = pd.read_csv(os.path.join(project_data_dir,'master_participants_data.csv'),index_col=0)

In [6]:
def glob_metric(hippunfold_dir,den='0p5mm',metric='thickness',space='T1w',label='hipp',ext='.dscalar.nii',desc=None,smooth='1mm'):
    
    if desc == None:
        paths = glob(os.path.join(hippunfold_dir, f'sub-*/surf/sub-*_space-{space}_den-{den}_label-{label}_{metric}{ext}'))
        if paths == []:
            paths = glob(os.path.join(hippunfold_dir, f'sub-*/ses-*1/surf/sub-*_ses-*1_space-{space}_den-{den}_label-{label}_{metric}{ext}'))
    else:
        paths = glob(os.path.join(hippunfold_dir, f'sub-*/surf/sub-*_space-{space}_den-{den}_label-{label}_desc-{desc}_smooth-{smooth}_{metric}{ext}'))
        if paths == []:
            paths = glob(os.path.join(hippunfold_dir, f'sub-*/ses-*1/surf/sub-*_ses-*1_space-{space}_den-{den}_label-{label}_desc-{desc}_smooth-{smooth}_{metric}{ext}'))
    
    try:
        paths = sorted(paths)
        subjects = [ p.split('/')[-1].split('_')[0] for p in paths]
    except:
        subjects = []
    
    return pd.DataFrame({'participant_id':subjects, 'paths':paths})

In [7]:
datasets_paths

{'aomic_piop1': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/AOMIC/ds002785',
 'aomic_piop2': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/AOMIC/ds002790',
 'aomic_id1000': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/AOMIC/ds003097',
 'cam-can': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/cam-can',
 'hcp1200': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/hcp1200',
 'hcp_aging': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/hcp_aging',
 'mpi_leipzig': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/mpi_leipzig',
 'neurocog_aging': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/neurocog_aging',
 'narratives': '/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/narratives',
 'qtim': '/home/

In [71]:
metrics = ['thickness','gyrification','myelin']
metric_csvs = []

for met_idx, met in enumerate(metrics):

    paths = []

    for dset_idx, dset in enumerate(datasets_paths):
        met_sub = glob_metric(hippunfold_path(datasets_paths[dset]),metric=met)
        
        if met_sub.size == 0:
            continue
        else:   
            met_sub['dataset'] = dset
            met_sub = pd.merge(met_sub,master_participant_df,on=['participant_id','dataset'],how='left')
            met_sub.dropna(inplace=True)
            met_sub.reset_index(drop=True,inplace=True)

            paths.append(met_sub[['participant_id','age','sex','site','dataset','paths']])
    
    metric_csvs.append(pd.concat(paths))


In [106]:
for met_idx, met in enumerate(metrics):
    metric_csvs[met_idx].reset_index(drop=True).to_csv(os.path.join(project_data_dir,f'{met}_participants_data.csv'))


In [113]:
os.path.exists(os.path.join(datasets_paths['cam-can'],'derivatives/hipposample'))

True

In [117]:
datasets_paths['aomic_piop1']

'/home/ROBARTS/myousif/graham/projects/ctb-akhanf/myousif9/norm_models/datasets/AOMIC/ds002785'

In [8]:
test_glob_dwi = glob_metric(os.path.join(datasets_paths['aomic_piop1'],'derivatives/hipposample'),metric='FA',desc='eddy')

In [ ]:
met_sub

In [10]:
metrics_diff = ['FA','MD']
metric_diff_csvs = []

for met_idx, met in enumerate(metrics_diff):

    paths = []

    for dset_idx, dset in enumerate(datasets_paths):
        met_sub = glob_metric(os.path.join(datasets_paths[dset],'derivatives/hipposample'),metric=met,desc='eddy')
        
        if met_sub.size == 0:
            continue
        else:   
            met_sub['dataset'] = dset
            met_sub = pd.merge(met_sub,master_participant_df,on=['participant_id','dataset'],how='left')
            met_sub.dropna(inplace=True)
            met_sub.reset_index(drop=True,inplace=True)

            paths.append(met_sub[['participant_id','age','sex','site','dataset','paths']])
    print(paths)
    metric_diff_csvs.append(pd.concat(paths))

[    participant_id    age sex         site      dataset  \
0         sub-0002  21.00   F  aomic_piop1  aomic_piop1   
1         sub-0003  23.00   M  aomic_piop1  aomic_piop1   
2         sub-0004  23.50   F  aomic_piop1  aomic_piop1   
3         sub-0005  21.75   M  aomic_piop1  aomic_piop1   
4         sub-0007  25.50   F  aomic_piop1  aomic_piop1   
..             ...    ...  ..          ...          ...   
116       sub-0209  23.00   F  aomic_piop1  aomic_piop1   
117       sub-0211  22.50   M  aomic_piop1  aomic_piop1   
118       sub-0213  19.75   M  aomic_piop1  aomic_piop1   
119       sub-0214  19.50   M  aomic_piop1  aomic_piop1   
120       sub-0216  20.50   F  aomic_piop1  aomic_piop1   

                                                 paths  
0    /home/ROBARTS/myousif/graham/projects/ctb-akha...  
1    /home/ROBARTS/myousif/graham/projects/ctb-akha...  
2    /home/ROBARTS/myousif/graham/projects/ctb-akha...  
3    /home/ROBARTS/myousif/graham/projects/ctb-akha...  
4    

In [37]:
for met_idx, met in enumerate(metrics_diff):
    metric_diff_csvs[met_idx].reset_index(drop=True).to_csv(os.path.join(project_data_dir,f'{met}_participants_data.csv'))